# **<font color='navy'> Classification of quantum channels</font>**
Here we propose a neural network-based approach for distinguishing between  single-qubit quantum channels.

# **<font color='navy'> Libraries and Random Seeds</font>**

In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import qiskit as qk
import qiskit.visualization

import qutip as qp

from qiskit import Aer
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector

from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit import transpile
from qiskit.quantum_info.operators import Operator


import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow.python.framework.ops import convert_to_tensor

TensorFlow version: 2.11.0


In [2]:
# Ensure reproducibility
np.random.seed(123)
tf.random.set_seed(123)
import random
random.seed(10)

# **<font color='navy'> Functions</font>**

In [12]:
def random_unitary(N):
    """
    Generation of an NxN random unitary matrix.
    """
    Z=np.random.randn(N,N) + 1.0j * np.random.randn(N,N)
    [Q,R]=sp.linalg.qr(Z)
    D=np.diag(np.diagonal(R)/np.abs(np.diagonal(R)))
    return np.dot(Q,D)

In [15]:
def get_density_matrix(state_vector):
    """
    Compute the density matrix from the state vector: 
       rho = |psi >< psi|.
    """
    density_matrix = np.outer(state_vector, np.conjugate(state_vector))
    return density_matrix

In [3]:
# Here we define the identity matrix and the Pauli matrices for dimension 2 (one qubit)

I = np.array([[1, 0],[0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Function which computes the components of the Bloch vector, given the density matrix 

def Bloch_vector(rho):
    """
    Compute the components of the Bloch vector, given the density matrix rho:
      r_i = Tr[rho * i].
    """
    ax = np.trace(np.dot(rho, X)).real
    ay = np.trace(np.dot(rho, Y)).real
    az = np.trace(np.dot(rho, Z)).real
    pnt = [ax, ay, az]
    return np.array(pnt)

In [4]:
def density_matrix_from_bloch_vector(bloch_vector):
    """
    Compute the density matrix given the Bloch vector:
    rho = 0.5 * (I + r_x * X + r_y * Y + r_z * Z).
    """
    rho = 0.5 * (I+ bloch_vector[0]*X + bloch_vector[1]*Y + bloch_vector[2]*Z)
    return rho

In [5]:
# Setting the simulator
sim_bknd=Aer.get_backend('aer_simulator')

In [6]:
def generate_Haar_data(num_qubits, samples=1000):
    """
    Generate "samples" Haar distributed data, for "num_qubits"- qubit states.
    """
    data = []
    for i in range(samples):
        qc = qk.QuantumCircuit(num_qubits) # create a quantum circuit with "num_qubits" qubits
        u = random_unitary(2**num_qubits)
        qc.unitary(u, qubits=range(num_qubits)) # apply the random unitary transformation to the circuit
        qc = qk.transpile(qc, backend=sim_bknd) # optimize the circuit
        qc.save_statevector() # save the state vector obtained by the simulation

        state = sim_bknd.run(qc).result().get_statevector(qc) # simulation and get the state vector
        state = np.asarray(state)
        data.append(state) 
    return data

# **<font color='navy'> Noise Models</font>**

In [7]:
# Single element of the operator-sum representation

def sum_element(rho,operator):
    """
    Single element of the operator-sum representation:
    e_i = E_i rho E_i^dagger.
    """
    element = np.dot(np.dot(operator,rho),operator.conj().T)
    return element

In [8]:
# Bit flip, Phase flip and Bit-Phase flip channels

def flip_error(num_qubits, rho, error_type, p):

    E_0 = np.sqrt(1-p)*I
    if error_type == 'bit':
        E_1 = np.sqrt(p)*X
    if error_type == 'phase':
        E_1 = np.sqrt(p)*Z
    if error_type == 'bp':
        E_1 = np.sqrt(p)*Y
        
    rho_with_flip_error = sum_element(rho, E_0)+sum_element(rho, E_1)
    
    return rho_with_flip_error

In [9]:
# Generalized Pauli channel

def g_pauli_error(num_qubits, rho, p_0, p_1, p_2, p_3):
    
    E_0 = np.sqrt(p_0)*I
    E_1 = np.sqrt(p_1)*X
    E_2 = np.sqrt(p_2)*Y
    E_3 = np.sqrt(p_3)*Z
    
    rho_with_gpauli_error = sum_element(rho, E_0)+sum_element(rho, E_1)+sum_element(rho, E_2)+sum_element(rho, E_3)
    
    return rho_with_gpauli_error

In [10]:
# Depolarizing channel

def depolar_error(num_qubits, rho, p):
    
    rho_with_dep_error = p*I/2 + (1-p)*rho

    return rho_with_dep_error

In [18]:
# Generalized Amplitude Damping channel

def ampl_damp_error(num_qubits, rho, p, a):
  
    E_0 = np.sqrt(p)*np.array([[1, 0], [0, np.sqrt(1-a)]])
    E_1 = np.sqrt(p)*np.array([[0, np.sqrt(a)], [0, 0]])
    E_2 = np.sqrt(1-p)*np.array([[np.sqrt(1-a), 0], [0, 1]])
    E_3 = np.sqrt(1-p)*np.array([[0, 0], [np.sqrt(a), 0]])
    
    rho_with_ad_error = sum_element(rho, E_0)+sum_element(rho, E_1)+sum_element(rho, E_2)+sum_element(rho, E_3)

    return rho_with_ad_error

# **<font color='navy'>Binary Classification: Phase vs Amplitude Damping</font>**
In this section, we construct a dataset which contains quantum states which can undergo into two different channels: phase flip (p=0.2) and amplitude damping channels (p=0.5, a= 0.3). We can see that, given a dataset of few hundreds of samples, the network correctly classifies all the samples. 
Notice that the dataset is composed by a six-component vector, representing the noisy Bloch vector appended to the noise-free one, and the label, in which 0 stands for phase flip channel while 1 for amplitude damping channel. The network reaches a validation accuracy of 1 with one hundred epochs, even if the loss function continues to decrease (making the decision process more reliable, increasing the probability of the correct class).

In [13]:
# Generate data
num_samples = 600
data = generate_Haar_data(1, num_samples)

In [16]:
# Compute noise-free density matrices and Bloch vectors
density_matrix_noise_free = [*map(get_density_matrix, data)]
bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

In [19]:
# Apply noise
density_matrix_with_noise = []
for i in range(num_samples):
  if (i%2==0): 
    single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
    density_matrix_with_noise.append(single_data_with_noise)
  else:
    single_data_with_noise = ampl_damp_error(1, density_matrix_noise_free[i], 0.5, 0.3)
    density_matrix_with_noise.append(single_data_with_noise)  

# Compute noisy Bloch vectors
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]

In [20]:
# Concatenating the noisy and noise-free Bloch vectors
x_dataset = []
for i in range(num_samples):
   element = np.concatenate((bloch_vectors_with_noise[i], bloch_vectors_noise_free[i]))
   x_dataset.append(element)

In [21]:
first_slice_value = 320
last_slice_value = 360

# Splitting the dataset
x_train_list, x_val_list, x_test_list = x_dataset[:300], x_dataset[300:400], x_dataset[400:500]

# Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
x_val = tf.convert_to_tensor(x_val_list)
x_test = tf.convert_to_tensor(x_test_list)

# Defining the labels (Phase Error = 0, Amplitude Damping Error = 1)
y_label_train = [0,1] * int(300/2)
y_label_val = [0,1] * int((100)/2)
y_label_test = [0,1] * int((100)/2)

# Convert to tensors
y_train = tf.convert_to_tensor(y_label_train)
y_val = tf.convert_to_tensor(y_label_val)
y_test = tf.convert_to_tensor(y_label_test)

# Define the one-hot vectors
y_train_oh = tf.keras.utils.to_categorical(y_train, 2)
y_val_oh = tf.keras.utils.to_categorical(y_val, 2)
y_test_oh = tf.keras.utils.to_categorical(y_test, 2)

In [22]:
# Define Model
model = tf.keras.models.Sequential([                               
  tf.keras.layers.InputLayer(input_shape=(6,)),                            
  tf.keras.layers.Dense(32, activation='relu'),  
  tf.keras.layers.Dense(16, activation='relu'),     
  tf.keras.layers.Dense(2, activation = 'softmax')
  ])

In [23]:
adam_opt = tf.optimizers.Adam(0.005)
# Compile model
model.compile(loss='categorical_crossentropy',
             optimizer=adam_opt,
             metrics=['accuracy'])

In [24]:
# Train Model
history = model.fit(x_train,
         y_train_oh,
         batch_size=50,
         epochs=300,
         validation_data=(x_val, y_val_oh))

Epoch 1/300
6/6 [==============================] - 3s 118ms/step - loss: 0.6963 - accuracy: 0.5333 - val_loss: 0.7004 - val_accuracy: 0.5300
Epoch 2/300
6/6 [==============================] - 0s 19ms/step - loss: 0.6750 - accuracy: 0.5467 - val_loss: 0.6973 - val_accuracy: 0.5000
Epoch 3/300
6/6 [==============================] - 0s 21ms/step - loss: 0.6683 - accuracy: 0.6000 - val_loss: 0.6953 - val_accuracy: 0.4900
Epoch 4/300
6/6 [==============================] - 0s 21ms/step - loss: 0.6611 - accuracy: 0.6133 - val_loss: 0.6972 - val_accuracy: 0.4600
Epoch 5/300
6/6 [==============================] - 0s 18ms/step - loss: 0.6535 - accuracy: 0.6133 - val_loss: 0.6980 - val_accuracy: 0.5000
Epoch 6/300
6/6 [==============================] - 0s 16ms/step - loss: 0.6475 - accuracy: 0.6267 - val_loss: 0.6932 - val_accuracy: 0.5100
Epoch 7/300
6/6 [==============================] - 0s 22ms/step - loss: 0.6379 - accuracy: 0.6233 - val_loss: 0.6761 - val_accuracy: 0.5400
Epoch 8/300
6/6 [==

6/6 [==============================] - 0s 19ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 60/300
6/6 [==============================] - 0s 22ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 61/300
6/6 [==============================] - 0s 21ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 62/300
6/6 [==============================] - 0s 22ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.0058 - val_accuracy: 1.0000
Epoch 63/300
6/6 [==============================] - 0s 22ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0056 - val_accuracy: 1.0000
Epoch 64/300
6/6 [==============================] - 0s 24ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 65/300
6/6 [==============================] - 0s 21ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 66/300
6/6 [========

Epoch 117/300
6/6 [==============================] - 0s 18ms/step - loss: 9.3685e-04 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 118/300
6/6 [==============================] - 0s 16ms/step - loss: 9.2382e-04 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 119/300
6/6 [==============================] - 0s 16ms/step - loss: 8.9643e-04 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 120/300
6/6 [==============================] - 0s 15ms/step - loss: 8.8224e-04 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 121/300
6/6 [==============================] - 0s 24ms/step - loss: 8.6437e-04 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 122/300
6/6 [==============================] - 0s 21ms/step - loss: 8.4798e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 123/300
6/6 [==============================] - 0s 18ms/step - loss: 8.3048e-04 - accuracy: 1.0000 - val_loss: 0.0011 -

Epoch 172/300
6/6 [==============================] - 0s 14ms/step - loss: 3.7962e-04 - accuracy: 1.0000 - val_loss: 5.1417e-04 - val_accuracy: 1.0000
Epoch 173/300
6/6 [==============================] - 0s 21ms/step - loss: 3.7469e-04 - accuracy: 1.0000 - val_loss: 5.0855e-04 - val_accuracy: 1.0000
Epoch 174/300
6/6 [==============================] - 0s 20ms/step - loss: 3.6963e-04 - accuracy: 1.0000 - val_loss: 4.9991e-04 - val_accuracy: 1.0000
Epoch 175/300
6/6 [==============================] - 0s 16ms/step - loss: 3.6572e-04 - accuracy: 1.0000 - val_loss: 4.9312e-04 - val_accuracy: 1.0000
Epoch 176/300
6/6 [==============================] - 0s 18ms/step - loss: 3.5951e-04 - accuracy: 1.0000 - val_loss: 4.8643e-04 - val_accuracy: 1.0000
Epoch 177/300
6/6 [==============================] - 0s 19ms/step - loss: 3.5478e-04 - accuracy: 1.0000 - val_loss: 4.8172e-04 - val_accuracy: 1.0000
Epoch 178/300
6/6 [==============================] - 0s 18ms/step - loss: 3.5098e-04 - accuracy: 1.0

6/6 [==============================] - 0s 18ms/step - loss: 2.0148e-04 - accuracy: 1.0000 - val_loss: 2.7552e-04 - val_accuracy: 1.0000
Epoch 227/300
6/6 [==============================] - 0s 18ms/step - loss: 1.9942e-04 - accuracy: 1.0000 - val_loss: 2.7311e-04 - val_accuracy: 1.0000
Epoch 228/300
6/6 [==============================] - 0s 16ms/step - loss: 1.9751e-04 - accuracy: 1.0000 - val_loss: 2.7133e-04 - val_accuracy: 1.0000
Epoch 229/300
6/6 [==============================] - 0s 20ms/step - loss: 1.9553e-04 - accuracy: 1.0000 - val_loss: 2.6856e-04 - val_accuracy: 1.0000
Epoch 230/300
6/6 [==============================] - 0s 16ms/step - loss: 1.9334e-04 - accuracy: 1.0000 - val_loss: 2.6563e-04 - val_accuracy: 1.0000
Epoch 231/300
6/6 [==============================] - 0s 18ms/step - loss: 1.9133e-04 - accuracy: 1.0000 - val_loss: 2.6277e-04 - val_accuracy: 1.0000
Epoch 232/300
6/6 [==============================] - 0s 19ms/step - loss: 1.8949e-04 - accuracy: 1.0000 - val_loss

Epoch 281/300
6/6 [==============================] - 0s 16ms/step - loss: 1.2035e-04 - accuracy: 1.0000 - val_loss: 1.6753e-04 - val_accuracy: 1.0000
Epoch 282/300
6/6 [==============================] - 0s 16ms/step - loss: 1.1936e-04 - accuracy: 1.0000 - val_loss: 1.6596e-04 - val_accuracy: 1.0000
Epoch 283/300
6/6 [==============================] - 0s 16ms/step - loss: 1.1839e-04 - accuracy: 1.0000 - val_loss: 1.6481e-04 - val_accuracy: 1.0000
Epoch 284/300
6/6 [==============================] - 0s 19ms/step - loss: 1.1738e-04 - accuracy: 1.0000 - val_loss: 1.6365e-04 - val_accuracy: 1.0000
Epoch 285/300
6/6 [==============================] - 0s 18ms/step - loss: 1.1635e-04 - accuracy: 1.0000 - val_loss: 1.6244e-04 - val_accuracy: 1.0000
Epoch 286/300
6/6 [==============================] - 0s 18ms/step - loss: 1.1541e-04 - accuracy: 1.0000 - val_loss: 1.6103e-04 - val_accuracy: 1.0000
Epoch 287/300
6/6 [==============================] - 0s 22ms/step - loss: 1.1449e-04 - accuracy: 1.0

In [25]:
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('Model Accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('Model Loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

In [26]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test_oh, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 1.0


In [27]:
#y_tilde = model.predict(x_test)
#for i in range(len(x_test)):
#  print(i, "Prediction: {}, True label: {}".format(y_tilde[i], y_test[i]))

# **<font color='navy'>Binary Classification: Amplitude Damping vs Depolarizing</font>**

In [28]:
# Generate data
num_samples = 2000
data = generate_Haar_data(1, num_samples)

In [29]:
# Compute noise-free density matrices and Bloch vectors
density_matrix_noise_free = [*map(get_density_matrix, data)]
bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

In [30]:
# Apply noise
density_matrix_with_noise = []
for i in range(num_samples):
  if (i%2==0): 
    single_data_with_noise = depolar_error(1, density_matrix_noise_free[i], 0.2)
    density_matrix_with_noise.append(single_data_with_noise)
  else:
    single_data_with_noise = ampl_damp_error(1, density_matrix_noise_free[i], 0.5, 0.3)
    density_matrix_with_noise.append(single_data_with_noise)  
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]

In [31]:
# Concatenating the noisy and noise-free Bloch vectors
x_dataset = []
for i in range(num_samples):
   element = np.concatenate((bloch_vectors_with_noise[i], bloch_vectors_noise_free[i]))
   x_dataset.append(element)

In [32]:
first_slice_value = 1600
last_slice_value = 1800

# Splitting the dataset
x_train_list, x_val_list, x_test_list = x_dataset[:first_slice_value], x_dataset[first_slice_value:last_slice_value], x_dataset[last_slice_value:]

# Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
x_val = tf.convert_to_tensor(x_val_list)
x_test = tf.convert_to_tensor(x_test_list)

# Defining the labels (Phase Error = 0, Amplitude Damping Error = 1)
y_label_train = [0,1] * int(first_slice_value/2)
y_label_val = [0,1] * int((last_slice_value - first_slice_value)/2)
y_label_test = [0,1] * int((last_slice_value - first_slice_value)/2)

# Convert to tensors
y_train = tf.convert_to_tensor(y_label_train)
y_val = tf.convert_to_tensor(y_label_val)
y_test = tf.convert_to_tensor(y_label_test)

# Define the one-hot vectors
y_train_oh = tf.keras.utils.to_categorical(y_train, 2)
y_val_oh = tf.keras.utils.to_categorical(y_val, 2)
y_test_oh = tf.keras.utils.to_categorical(y_test, 2)

In [33]:
# Define Model
model = tf.keras.models.Sequential([                                                         
  tf.keras.layers.Dense(32, activation='relu'),  
  tf.keras.layers.Dropout(0.2),
  #tf.keras.layers.Dense(32, activation='relu'),  
  tf.keras.layers.Dense(16, activation='relu'),     
  tf.keras.layers.Dense(2, activation = 'softmax')
  ])

In [34]:
adam_opt = tf.optimizers.Adam(0.001)
# Compile model
model.compile(loss='categorical_crossentropy',
             optimizer=adam_opt,
             metrics=['accuracy'])

In [35]:
# Train Model
history = model.fit(x_train,
         y_train_oh,
         batch_size=150,
         epochs=1000,
         validation_data=(x_val, y_val_oh))

Epoch 1/1000
11/11 [==============================] - 2s 60ms/step - loss: 0.7123 - accuracy: 0.4938 - val_loss: 0.6996 - val_accuracy: 0.4850
Epoch 2/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.7000 - accuracy: 0.5113 - val_loss: 0.6941 - val_accuracy: 0.4900
Epoch 3/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.6968 - accuracy: 0.4975 - val_loss: 0.6928 - val_accuracy: 0.5200
Epoch 4/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.6947 - accuracy: 0.5038 - val_loss: 0.6930 - val_accuracy: 0.5150
Epoch 5/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.6950 - accuracy: 0.4975 - val_loss: 0.6929 - val_accuracy: 0.4950
Epoch 6/1000
11/11 [==============================] - 0s 17ms/step - loss: 0.6942 - accuracy: 0.5125 - val_loss: 0.6925 - val_accuracy: 0.5150
Epoch 7/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.6932 - accuracy: 0.5306 - val_loss: 0.6926 - val_accuracy: 0.4950

Epoch 58/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.6793 - accuracy: 0.5844 - val_loss: 0.6848 - val_accuracy: 0.5650
Epoch 59/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.6810 - accuracy: 0.5675 - val_loss: 0.6836 - val_accuracy: 0.5550
Epoch 60/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.6821 - accuracy: 0.5694 - val_loss: 0.6838 - val_accuracy: 0.5700
Epoch 61/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.6790 - accuracy: 0.5744 - val_loss: 0.6837 - val_accuracy: 0.5750
Epoch 62/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.6782 - accuracy: 0.5819 - val_loss: 0.6830 - val_accuracy: 0.5550
Epoch 63/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.6796 - accuracy: 0.5819 - val_loss: 0.6822 - val_accuracy: 0.5750
Epoch 64/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.6785 - accuracy: 0.5863 - val_loss: 0.6817 - val_accuracy:

Epoch 115/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.5389 - accuracy: 0.8031 - val_loss: 0.5496 - val_accuracy: 0.8550
Epoch 116/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.5296 - accuracy: 0.8163 - val_loss: 0.5389 - val_accuracy: 0.8750
Epoch 117/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.5279 - accuracy: 0.8081 - val_loss: 0.5359 - val_accuracy: 0.8700
Epoch 118/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.5114 - accuracy: 0.8306 - val_loss: 0.5232 - val_accuracy: 0.9000
Epoch 119/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.5090 - accuracy: 0.8156 - val_loss: 0.5177 - val_accuracy: 0.8750
Epoch 120/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.4959 - accuracy: 0.8319 - val_loss: 0.5122 - val_accuracy: 0.8900
Epoch 121/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.4916 - accuracy: 0.8313 - val_loss: 0.5069 - val_ac

11/11 [==============================] - 0s 12ms/step - loss: 0.1760 - accuracy: 0.9812 - val_loss: 0.1934 - val_accuracy: 1.0000
Epoch 172/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.1801 - accuracy: 0.9787 - val_loss: 0.1885 - val_accuracy: 0.9950
Epoch 173/1000
11/11 [==============================] - 0s 16ms/step - loss: 0.1710 - accuracy: 0.9819 - val_loss: 0.1831 - val_accuracy: 0.9950
Epoch 174/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.1676 - accuracy: 0.9775 - val_loss: 0.1834 - val_accuracy: 0.9900
Epoch 175/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.1692 - accuracy: 0.9762 - val_loss: 0.1848 - val_accuracy: 0.9950
Epoch 176/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.1695 - accuracy: 0.9750 - val_loss: 0.1825 - val_accuracy: 0.9800
Epoch 177/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.1696 - accuracy: 0.9694 - val_loss: 0.1772 - val_accuracy: 0.9950


11/11 [==============================] - 0s 11ms/step - loss: 0.0899 - accuracy: 0.9937 - val_loss: 0.0935 - val_accuracy: 1.0000
Epoch 228/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.0856 - accuracy: 0.9912 - val_loss: 0.0928 - val_accuracy: 1.0000
Epoch 229/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0797 - accuracy: 0.9937 - val_loss: 0.0905 - val_accuracy: 0.9950
Epoch 230/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0823 - accuracy: 0.9944 - val_loss: 0.0890 - val_accuracy: 1.0000
Epoch 231/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0859 - accuracy: 0.9912 - val_loss: 0.0883 - val_accuracy: 1.0000
Epoch 232/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.0813 - accuracy: 0.9931 - val_loss: 0.0912 - val_accuracy: 0.9950
Epoch 233/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0783 - accuracy: 0.9931 - val_loss: 0.0855 - val_accuracy: 1.0000


...

11/11 [==============================] - 0s 11ms/step - loss: 0.0076 - accuracy: 0.9987 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 732/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0070 - accuracy: 0.9987 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 733/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0081 - accuracy: 0.9969 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 734/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 735/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.0077 - accuracy: 0.9994 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 736/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.0056 - accuracy: 0.9994 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 737/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0077 - accuracy: 0.9987 - val_loss: 0.0026 - val_accuracy: 1.0000


11/11 [==============================] - 0s 12ms/step - loss: 0.0076 - accuracy: 0.9981 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 788/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0065 - accuracy: 0.9987 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 789/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0043 - accuracy: 0.9994 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 790/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0054 - accuracy: 0.9987 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 791/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.0057 - accuracy: 0.9987 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 792/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.0060 - accuracy: 0.9987 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 793/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.0050 - accuracy: 0.9994 - val_loss: 0.0021 - val_accuracy: 1.0000


11/11 [==============================] - 0s 11ms/step - loss: 0.0058 - accuracy: 0.9994 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 844/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.0049 - accuracy: 0.9987 - val_loss: 0.0023 - val_accuracy: 1.0000
Epoch 845/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.0081 - accuracy: 0.9981 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 846/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0085 - accuracy: 0.9975 - val_loss: 0.0050 - val_accuracy: 1.0000
Epoch 847/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0062 - accuracy: 0.9981 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 848/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0077 - accuracy: 0.9987 - val_loss: 0.0030 - val_accuracy: 1.0000
Epoch 849/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.0020 - val_accuracy: 1.0000


11/11 [==============================] - 0s 12ms/step - loss: 0.0067 - accuracy: 0.9981 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 900/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0084 - accuracy: 0.9981 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 901/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0049 - accuracy: 0.9987 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 902/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0081 - accuracy: 0.9969 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 903/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0079 - accuracy: 0.9969 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 904/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0086 - accuracy: 0.9975 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 905/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0085 - accuracy: 0.9962 - val_loss: 0.0022 - val_accuracy: 1.0000


11/11 [==============================] - 0s 11ms/step - loss: 0.0041 - accuracy: 0.9981 - val_loss: 0.0034 - val_accuracy: 1.0000
Epoch 956/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0034 - accuracy: 0.9994 - val_loss: 8.3583e-04 - val_accuracy: 1.0000
Epoch 957/1000
11/11 [==============================] - 0s 11ms/step - loss: 0.0063 - accuracy: 0.9975 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 958/1000
11/11 [==============================] - 0s 13ms/step - loss: 0.0056 - accuracy: 0.9987 - val_loss: 9.1227e-04 - val_accuracy: 1.0000
Epoch 959/1000
11/11 [==============================] - 0s 14ms/step - loss: 0.0062 - accuracy: 0.9981 - val_loss: 7.2608e-04 - val_accuracy: 1.0000
Epoch 960/1000
11/11 [==============================] - 0s 10ms/step - loss: 0.0053 - accuracy: 0.9994 - val_loss: 5.9828e-04 - val_accuracy: 1.0000
Epoch 961/1000
11/11 [==============================] - 0s 12ms/step - loss: 0.0059 - accuracy: 0.9981 - val_loss: 0.0011 - val_a

In [36]:
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('Model Accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('Model Loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

In [37]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test_oh, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 1.0


In [38]:
#y_tilde = model.predict(x_test)
#for i in range(len(x_test)):
#  print(i, "Prediction: {}, True label: {}".format(y_tilde[i], y_test[i]))

# **<font color='navy'>Binary Classification with noisy dataset</font>**

In this section, we propose again the problem of binary classification but with a variation: in this case the dataset is only composed by the noisy Bloch vector. The network has to investigate hidden relationships among the training data and the corresponding effect on the Bloch sphere. We expect the network to learn the different transformations in which the Bloch sphere goes when a particular error is applied. In fact, for the phase flip channel the x-y plane of Bloch sphere is uniformly contracted by a factor *1-2p*, while the states of the z-axis are left alone. Instead, for the generalized amplitude damping channel, the sphere shrinks towards a given point (determined by the parameter a) along the z-axis. 

We can see that the amount of data required for this task is higher than the previous case because we have provided less informative data to the network. Moreover, the network doesn't reach accuracy 1 and it makes sense: some samples could belong reasonably to both classes and the model can't decide how to classify them (the two different Bloch spheres have a not trivial intersection); this also explains the instability of the evaluation curves in trying to classify ambiguous samples. 

Finally, also the network must be more complex: more hidden layers are required.

In [39]:
# Generate data
num_samples = 3000
data = generate_Haar_data(1, num_samples)

In [40]:
# Compute noise-free density matrices and Bloch vectors
density_matrix_noise_free = [*map(get_density_matrix, data)]
bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

In [41]:
# Apply noise
density_matrix_with_noise = []
den_ad_with_noise = []
den_pf_with_noise = []
for i in range(num_samples):
  if (i%2==0): 
    single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
    density_matrix_with_noise.append(single_data_with_noise)
    den_pf_with_noise.append(single_data_with_noise)
  else:
    single_data_with_noise = ampl_damp_error(1, density_matrix_noise_free[i], 0.5, 0.3)
    density_matrix_with_noise.append(single_data_with_noise) 
    den_ad_with_noise.append(single_data_with_noise)

# Compute the noisy Bloch vectors
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
blo_vectors_with_noise = [*map(Bloch_vector, den_ad_with_noise)]
b_vec_with_noise = [*map(Bloch_vector, den_pf_with_noise)]

In [42]:
first_slice_value = 4000
last_slice_value = 4500

# Splitting the dataset
x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:800], bloch_vectors_with_noise[2500:2750], bloch_vectors_with_noise[2750:]

# Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
x_val = tf.convert_to_tensor(x_val_list)
x_test = tf.convert_to_tensor(x_test_list)

# Defining the labels (Phase Error = 0, Amplitude Damping Error = 1)
y_label_train = [0,1] * int(800/2)
y_label_val = [0,1] * int((250)/2)
y_label_test = [0,1] * int((250)/2)

# Convert to tensors
y_train = tf.convert_to_tensor(y_label_train)
y_val = tf.convert_to_tensor(y_label_val)
y_test = tf.convert_to_tensor(y_label_test)

# Define the one-hot vectors
y_train_oh = tf.keras.utils.to_categorical(y_train, 2)
y_val_oh = tf.keras.utils.to_categorical(y_val, 2)
y_test_oh = tf.keras.utils.to_categorical(y_test, 2)

In [43]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(3,)),                                 
  tf.keras.layers.Dense(128, activation='relu'), 
  tf.keras.layers.Dropout(0.1), 
  tf.keras.layers.Dense(64, activation='relu'),  
  tf.keras.layers.Dense(2, activation = 'softmax')
  ])

In [44]:
# Compile model
adam_opt = tf.optimizers.Adam(0.001)  
model.compile(loss='categorical_crossentropy',
             optimizer=adam_opt,
             metrics=['accuracy'])

In [45]:
# Train Model
batch_size = 30
epochs = 500

history = model.fit(x_train, y_train_oh, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val_oh))

Epoch 1/500
27/27 [==============================] - 2s 20ms/step - loss: 0.6845 - accuracy: 0.5550 - val_loss: 0.6674 - val_accuracy: 0.6160
Epoch 2/500
27/27 [==============================] - 0s 9ms/step - loss: 0.6563 - accuracy: 0.6388 - val_loss: 0.6496 - val_accuracy: 0.6280
Epoch 3/500
27/27 [==============================] - 0s 9ms/step - loss: 0.6370 - accuracy: 0.6500 - val_loss: 0.6346 - val_accuracy: 0.6440
Epoch 4/500
27/27 [==============================] - 0s 9ms/step - loss: 0.6254 - accuracy: 0.6313 - val_loss: 0.6254 - val_accuracy: 0.6440
Epoch 5/500
27/27 [==============================] - 0s 9ms/step - loss: 0.6152 - accuracy: 0.6425 - val_loss: 0.6219 - val_accuracy: 0.6360
Epoch 6/500
27/27 [==============================] - 0s 9ms/step - loss: 0.6083 - accuracy: 0.6538 - val_loss: 0.6170 - val_accuracy: 0.6440
Epoch 7/500
27/27 [==============================] - 0s 9ms/step - loss: 0.6010 - accuracy: 0.6475 - val_loss: 0.6197 - val_accuracy: 0.6600
Epoch 8/500


Epoch 59/500
27/27 [==============================] - 0s 8ms/step - loss: 0.2420 - accuracy: 0.8725 - val_loss: 0.2579 - val_accuracy: 0.8600
Epoch 60/500
27/27 [==============================] - 0s 7ms/step - loss: 0.2430 - accuracy: 0.8813 - val_loss: 0.2573 - val_accuracy: 0.8640
Epoch 61/500
27/27 [==============================] - 0s 7ms/step - loss: 0.2353 - accuracy: 0.8825 - val_loss: 0.2489 - val_accuracy: 0.8600
Epoch 62/500
27/27 [==============================] - 0s 7ms/step - loss: 0.2331 - accuracy: 0.8863 - val_loss: 0.2465 - val_accuracy: 0.8680
Epoch 63/500
27/27 [==============================] - 0s 8ms/step - loss: 0.2318 - accuracy: 0.8863 - val_loss: 0.2460 - val_accuracy: 0.8560
Epoch 64/500
27/27 [==============================] - 0s 8ms/step - loss: 0.2203 - accuracy: 0.8850 - val_loss: 0.2394 - val_accuracy: 0.8920
Epoch 65/500
27/27 [==============================] - 0s 8ms/step - loss: 0.2309 - accuracy: 0.8800 - val_loss: 0.2383 - val_accuracy: 0.8840
Epoch 

27/27 [==============================] - 0s 7ms/step - loss: 0.1257 - accuracy: 0.9500 - val_loss: 0.1174 - val_accuracy: 0.9480
Epoch 117/500
27/27 [==============================] - 0s 9ms/step - loss: 0.1128 - accuracy: 0.9613 - val_loss: 0.1157 - val_accuracy: 0.9520
Epoch 118/500
27/27 [==============================] - 0s 8ms/step - loss: 0.1283 - accuracy: 0.9488 - val_loss: 0.1130 - val_accuracy: 0.9400
Epoch 119/500
27/27 [==============================] - 0s 7ms/step - loss: 0.1176 - accuracy: 0.9613 - val_loss: 0.1172 - val_accuracy: 0.9480
Epoch 120/500
27/27 [==============================] - 0s 8ms/step - loss: 0.1171 - accuracy: 0.9625 - val_loss: 0.1210 - val_accuracy: 0.9440
Epoch 121/500
27/27 [==============================] - 0s 9ms/step - loss: 0.1189 - accuracy: 0.9588 - val_loss: 0.1195 - val_accuracy: 0.9600
Epoch 122/500
27/27 [==============================] - 0s 8ms/step - loss: 0.1174 - accuracy: 0.9563 - val_loss: 0.1233 - val_accuracy: 0.9360
Epoch 123/500

27/27 [==============================] - 0s 8ms/step - loss: 0.0905 - accuracy: 0.9675 - val_loss: 0.0909 - val_accuracy: 0.9600
Epoch 174/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0830 - accuracy: 0.9712 - val_loss: 0.0959 - val_accuracy: 0.9560
Epoch 175/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0800 - accuracy: 0.9725 - val_loss: 0.0847 - val_accuracy: 0.9640
Epoch 176/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0750 - accuracy: 0.9787 - val_loss: 0.0972 - val_accuracy: 0.9440
Epoch 177/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0782 - accuracy: 0.9737 - val_loss: 0.1028 - val_accuracy: 0.9440
Epoch 178/500
27/27 [==============================] - 0s 9ms/step - loss: 0.0793 - accuracy: 0.9750 - val_loss: 0.0894 - val_accuracy: 0.9520
Epoch 179/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0822 - accuracy: 0.9737 - val_loss: 0.0897 - val_accuracy: 0.9520
Epoch 180/500

27/27 [==============================] - 0s 9ms/step - loss: 0.0663 - accuracy: 0.9725 - val_loss: 0.0987 - val_accuracy: 0.9440
Epoch 231/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0635 - accuracy: 0.9750 - val_loss: 0.0929 - val_accuracy: 0.9480
Epoch 232/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0778 - accuracy: 0.9700 - val_loss: 0.0831 - val_accuracy: 0.9640
Epoch 233/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0747 - accuracy: 0.9737 - val_loss: 0.0952 - val_accuracy: 0.9480
Epoch 234/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0615 - accuracy: 0.9762 - val_loss: 0.0817 - val_accuracy: 0.9600
Epoch 235/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0678 - accuracy: 0.9800 - val_loss: 0.0812 - val_accuracy: 0.9600
Epoch 236/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0722 - accuracy: 0.9675 - val_loss: 0.0879 - val_accuracy: 0.9560
Epoch 237/500

27/27 [==============================] - 0s 8ms/step - loss: 0.0643 - accuracy: 0.9712 - val_loss: 0.0875 - val_accuracy: 0.9600
Epoch 288/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0627 - accuracy: 0.9762 - val_loss: 0.0933 - val_accuracy: 0.9560
Epoch 289/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0498 - accuracy: 0.9800 - val_loss: 0.1088 - val_accuracy: 0.9400
Epoch 290/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0546 - accuracy: 0.9787 - val_loss: 0.0812 - val_accuracy: 0.9680
Epoch 291/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0515 - accuracy: 0.9800 - val_loss: 0.0939 - val_accuracy: 0.9600
Epoch 292/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0569 - accuracy: 0.9750 - val_loss: 0.0896 - val_accuracy: 0.9560
Epoch 293/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0554 - accuracy: 0.9775 - val_loss: 0.0969 - val_accuracy: 0.9560
Epoch 294/500

27/27 [==============================] - 0s 10ms/step - loss: 0.0492 - accuracy: 0.9812 - val_loss: 0.1041 - val_accuracy: 0.9520
Epoch 345/500
27/27 [==============================] - 0s 9ms/step - loss: 0.0390 - accuracy: 0.9900 - val_loss: 0.0868 - val_accuracy: 0.9600
Epoch 346/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0472 - accuracy: 0.9800 - val_loss: 0.0880 - val_accuracy: 0.9600
Epoch 347/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0480 - accuracy: 0.9812 - val_loss: 0.1003 - val_accuracy: 0.9640
Epoch 348/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0560 - accuracy: 0.9775 - val_loss: 0.1035 - val_accuracy: 0.9640
Epoch 349/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0452 - accuracy: 0.9862 - val_loss: 0.0927 - val_accuracy: 0.9520
Epoch 350/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0437 - accuracy: 0.9850 - val_loss: 0.0883 - val_accuracy: 0.9640
Epoch 351/50

27/27 [==============================] - 0s 8ms/step - loss: 0.0482 - accuracy: 0.9800 - val_loss: 0.1123 - val_accuracy: 0.9600
Epoch 402/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0537 - accuracy: 0.9800 - val_loss: 0.1150 - val_accuracy: 0.9560
Epoch 403/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0461 - accuracy: 0.9800 - val_loss: 0.1200 - val_accuracy: 0.9480
Epoch 404/500
27/27 [==============================] - 0s 9ms/step - loss: 0.0512 - accuracy: 0.9800 - val_loss: 0.0955 - val_accuracy: 0.9640
Epoch 405/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0459 - accuracy: 0.9812 - val_loss: 0.1024 - val_accuracy: 0.9560
Epoch 406/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0487 - accuracy: 0.9862 - val_loss: 0.1252 - val_accuracy: 0.9560
Epoch 407/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0510 - accuracy: 0.9825 - val_loss: 0.1012 - val_accuracy: 0.9600
Epoch 408/500

27/27 [==============================] - 0s 8ms/step - loss: 0.0418 - accuracy: 0.9837 - val_loss: 0.0905 - val_accuracy: 0.9520
Epoch 459/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0455 - accuracy: 0.9812 - val_loss: 0.1286 - val_accuracy: 0.9520
Epoch 460/500
27/27 [==============================] - 0s 7ms/step - loss: 0.0489 - accuracy: 0.9775 - val_loss: 0.0928 - val_accuracy: 0.9600
Epoch 461/500
27/27 [==============================] - 0s 9ms/step - loss: 0.0398 - accuracy: 0.9825 - val_loss: 0.1193 - val_accuracy: 0.9480
Epoch 462/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0371 - accuracy: 0.9850 - val_loss: 0.0959 - val_accuracy: 0.9640
Epoch 463/500
27/27 [==============================] - 0s 8ms/step - loss: 0.0429 - accuracy: 0.9862 - val_loss: 0.1093 - val_accuracy: 0.9600
Epoch 464/500
27/27 [==============================] - 0s 10ms/step - loss: 0.0368 - accuracy: 0.9850 - val_loss: 0.1034 - val_accuracy: 0.9520
Epoch 465/50

In [46]:
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

In [47]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test_oh, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.9520000219345093


In [48]:
#y_tilde = model.predict(x_test)
#for i in range(len(x_test)):
#  print(i, "Prediction: {}, True label: {}".format(y_tilde[i], y_test[i]))

# **<font color='navy'>Phase Flip vs Depolarizing vs Amplitude Damping</font>**

In [49]:
# Generate data
num_samples = 1200
data = generate_Haar_data(1, num_samples)

In [50]:
# Compute the noise-free density matrices and Bloch vectors
density_matrix_noise_free = [*map(get_density_matrix, data)]
bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

In [51]:
# Apply noise 
density_matrix_with_noise = []
for i in range(400):
   single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
   density_matrix_with_noise.append(single_data_with_noise)

for i in range(400, 800):
   single_data_with_noise = depolar_error(1, density_matrix_noise_free[i], 0.2)
   density_matrix_with_noise.append(single_data_with_noise)

for i in range(800, num_samples):
    single_data_with_noise = ampl_damp_error(1, density_matrix_noise_free[i], 0.5, 0.3)
    density_matrix_with_noise.append(single_data_with_noise)

# Compute noisy Bloch vectors   
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]

# Concatenate the noisy and the noise-free Bloch vectors
complete_vectors = []
for i in range(num_samples):
   element = np.concatenate((bloch_vectors_with_noise[i], bloch_vectors_noise_free[i]))
   complete_vectors.append(element)

for i in range(400):
  complete_vectors[i] = np.append(complete_vectors[i], 0) # phase flip channel

for i in range(400, 800):
  complete_vectors[i] = np.append(complete_vectors[i], 1) # depolarizing channel

for i in range(800, num_samples):
  complete_vectors[i] = np.append(complete_vectors[i], 2)  # amplitude damping channel

random.shuffle(complete_vectors)
#print(complete_vectors[3])

In [52]:
# Building the dataset

x_dataset = []

for item in complete_vectors:
  element = item[:-1]
  x_dataset.append(element)

#print(x_dataset)

y_dataset = []
for item in complete_vectors:
  element = item[-1:]
  y_dataset.append(element)

#print(y_dataset)

first_slice_value = 960
last_slice_value = 1080

# Splitting the dataset
x_train_list, x_val_list, x_test_list = x_dataset[:first_slice_value], x_dataset[first_slice_value:last_slice_value], x_dataset[last_slice_value:]

# Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
x_val = tf.convert_to_tensor(x_val_list)
x_test = tf.convert_to_tensor(x_test_list)

# Defining the labels 
y_label_train, y_label_val, y_label_test = y_dataset[:first_slice_value], y_dataset[first_slice_value:last_slice_value], y_dataset[last_slice_value:]

# Convert to tensors
y_train = tf.convert_to_tensor(y_label_train)
y_val = tf.convert_to_tensor(y_label_val)
y_test = tf.convert_to_tensor(y_label_test)

# Define the one-hot vectors
y_train_oh = tf.keras.utils.to_categorical(y_train, 3)
y_val_oh = tf.keras.utils.to_categorical(y_val, 3)
y_test_oh = tf.keras.utils.to_categorical(y_test, 3)

In [53]:
# Define Model
model = tf.keras.models.Sequential([  
  tf.keras.layers.InputLayer(input_shape=(6,)),              
  tf.keras.layers.Dense(128, activation='relu'),  
  #tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(128, activation='relu'), 
  tf.keras.layers.Dense(128, activation='relu'),  
  tf.keras.layers.Dense(3, activation = 'softmax')
  ])

In [54]:
# Compile model
adam_opt = tf.optimizers.Adam(0.0005)
model.compile(loss='categorical_crossentropy',
             optimizer=adam_opt,
             metrics=['accuracy'])

In [55]:
# Train Model
batch_size = 240
epochs = 1500 #with 300 epochs we can notice the difference between phase error and the other two

history = model.fit(x_train,
         y_train_oh,
         batch_size=batch_size,
         epochs=epochs,
         validation_data=(x_val, y_val_oh))

Epoch 1/1500
4/4 [==============================] - 2s 134ms/step - loss: 1.0996 - accuracy: 0.3208 - val_loss: 1.0928 - val_accuracy: 0.3833
Epoch 2/1500
4/4 [==============================] - 0s 32ms/step - loss: 1.0921 - accuracy: 0.3667 - val_loss: 1.0883 - val_accuracy: 0.3833
Epoch 3/1500
4/4 [==============================] - 0s 27ms/step - loss: 1.0867 - accuracy: 0.4375 - val_loss: 1.0844 - val_accuracy: 0.4667
Epoch 4/1500
4/4 [==============================] - 0s 27ms/step - loss: 1.0821 - accuracy: 0.5135 - val_loss: 1.0811 - val_accuracy: 0.4333
Epoch 5/1500
4/4 [==============================] - 0s 29ms/step - loss: 1.0772 - accuracy: 0.5010 - val_loss: 1.0777 - val_accuracy: 0.4667
Epoch 6/1500
4/4 [==============================] - 0s 29ms/step - loss: 1.0730 - accuracy: 0.5010 - val_loss: 1.0750 - val_accuracy: 0.4750
Epoch 7/1500
4/4 [==============================] - 0s 29ms/step - loss: 1.0684 - accuracy: 0.4917 - val_loss: 1.0708 - val_accuracy: 0.4917
Epoch 8/1500

Epoch 59/1500
4/4 [==============================] - 0s 35ms/step - loss: 0.2398 - accuracy: 0.9854 - val_loss: 0.2976 - val_accuracy: 0.9500
Epoch 60/1500
4/4 [==============================] - 0s 35ms/step - loss: 0.2344 - accuracy: 0.9875 - val_loss: 0.2803 - val_accuracy: 0.9583
Epoch 61/1500
4/4 [==============================] - 0s 32ms/step - loss: 0.2255 - accuracy: 0.9917 - val_loss: 0.2750 - val_accuracy: 0.9500
Epoch 62/1500
4/4 [==============================] - 0s 32ms/step - loss: 0.2159 - accuracy: 0.9844 - val_loss: 0.2642 - val_accuracy: 0.9583
Epoch 63/1500
4/4 [==============================] - 0s 32ms/step - loss: 0.2051 - accuracy: 0.9906 - val_loss: 0.2643 - val_accuracy: 0.9667
Epoch 64/1500
4/4 [==============================] - 0s 45ms/step - loss: 0.1975 - accuracy: 0.9906 - val_loss: 0.2437 - val_accuracy: 0.9750
Epoch 65/1500
4/4 [==============================] - 0s 29ms/step - loss: 0.1926 - accuracy: 0.9917 - val_loss: 0.2422 - val_accuracy: 0.9833
Epoch 

4/4 [==============================] - 0s 27ms/step - loss: 0.0329 - accuracy: 1.0000 - val_loss: 0.0607 - val_accuracy: 0.9917
Epoch 117/1500
4/4 [==============================] - 0s 27ms/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 0.0591 - val_accuracy: 1.0000
Epoch 118/1500
4/4 [==============================] - 0s 32ms/step - loss: 0.0316 - accuracy: 1.0000 - val_loss: 0.0567 - val_accuracy: 1.0000
Epoch 119/1500
4/4 [==============================] - 0s 24ms/step - loss: 0.0308 - accuracy: 1.0000 - val_loss: 0.0549 - val_accuracy: 1.0000
Epoch 120/1500
4/4 [==============================] - 0s 27ms/step - loss: 0.0302 - accuracy: 1.0000 - val_loss: 0.0579 - val_accuracy: 1.0000
Epoch 121/1500
4/4 [==============================] - 0s 32ms/step - loss: 0.0297 - accuracy: 1.0000 - val_loss: 0.0530 - val_accuracy: 1.0000
Epoch 122/1500
4/4 [==============================] - 0s 24ms/step - loss: 0.0285 - accuracy: 1.0000 - val_loss: 0.0543 - val_accuracy: 1.0000
Epoch 123/1500

4/4 [==============================] - 0s 27ms/step - loss: 0.0110 - accuracy: 1.0000 - val_loss: 0.0254 - val_accuracy: 1.0000
Epoch 174/1500
4/4 [==============================] - 0s 30ms/step - loss: 0.0113 - accuracy: 1.0000 - val_loss: 0.0246 - val_accuracy: 1.0000
Epoch 175/1500
4/4 [==============================] - 0s 27ms/step - loss: 0.0110 - accuracy: 1.0000 - val_loss: 0.0240 - val_accuracy: 1.0000
Epoch 176/1500
4/4 [==============================] - 0s 27ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 0.0253 - val_accuracy: 1.0000
Epoch 177/1500
4/4 [==============================] - 0s 29ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 0.0232 - val_accuracy: 1.0000
Epoch 178/1500
4/4 [==============================] - 0s 27ms/step - loss: 0.0106 - accuracy: 1.0000 - val_loss: 0.0234 - val_accuracy: 1.0000
Epoch 179/1500
4/4 [==============================] - 0s 32ms/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.0239 - val_accuracy: 1.0000
Epoch 180/1500

4/4 [==============================] - 0s 27ms/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 0.0129 - val_accuracy: 1.0000
Epoch 231/1500
4/4 [==============================] - 0s 37ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0136 - val_accuracy: 1.0000
Epoch 232/1500
4/4 [==============================] - 0s 32ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0130 - val_accuracy: 1.0000
Epoch 233/1500
4/4 [==============================] - 0s 24ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.0134 - val_accuracy: 1.0000
Epoch 234/1500
4/4 [==============================] - 0s 35ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0130 - val_accuracy: 1.0000
Epoch 235/1500
4/4 [==============================] - 0s 32ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0132 - val_accuracy: 1.0000
Epoch 236/1500
4/4 [==============================] - 0s 36ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0125 - val_accuracy: 1.0000
Epoch 237/1500

...

4/4 [==============================] - 0s 24ms/step - loss: 4.4343e-05 - accuracy: 1.0000 - val_loss: 3.7455e-04 - val_accuracy: 1.0000
Epoch 1282/1500
4/4 [==============================] - 0s 27ms/step - loss: 4.4373e-05 - accuracy: 1.0000 - val_loss: 3.7581e-04 - val_accuracy: 1.0000
Epoch 1283/1500
4/4 [==============================] - 0s 29ms/step - loss: 4.4307e-05 - accuracy: 1.0000 - val_loss: 3.9281e-04 - val_accuracy: 1.0000
Epoch 1284/1500
4/4 [==============================] - 0s 29ms/step - loss: 4.4116e-05 - accuracy: 1.0000 - val_loss: 3.8426e-04 - val_accuracy: 1.0000
Epoch 1285/1500
4/4 [==============================] - 0s 29ms/step - loss: 4.3848e-05 - accuracy: 1.0000 - val_loss: 3.7510e-04 - val_accuracy: 1.0000
Epoch 1286/1500
4/4 [==============================] - 0s 29ms/step - loss: 4.3809e-05 - accuracy: 1.0000 - val_loss: 3.7804e-04 - val_accuracy: 1.0000
Epoch 1287/1500
4/4 [==============================] - 0s 24ms/step - loss: 4.3794e-05 - accuracy: 1.000

4/4 [==============================] - 0s 32ms/step - loss: 3.8506e-05 - accuracy: 1.0000 - val_loss: 3.4830e-04 - val_accuracy: 1.0000
Epoch 1336/1500
4/4 [==============================] - 0s 29ms/step - loss: 3.8336e-05 - accuracy: 1.0000 - val_loss: 3.4672e-04 - val_accuracy: 1.0000
Epoch 1337/1500
4/4 [==============================] - 0s 27ms/step - loss: 3.8345e-05 - accuracy: 1.0000 - val_loss: 3.3690e-04 - val_accuracy: 1.0000
Epoch 1338/1500
4/4 [==============================] - 0s 27ms/step - loss: 3.7941e-05 - accuracy: 1.0000 - val_loss: 3.5341e-04 - val_accuracy: 1.0000
Epoch 1339/1500
4/4 [==============================] - 0s 30ms/step - loss: 3.7683e-05 - accuracy: 1.0000 - val_loss: 3.5136e-04 - val_accuracy: 1.0000
Epoch 1340/1500
4/4 [==============================] - 0s 27ms/step - loss: 3.8118e-05 - accuracy: 1.0000 - val_loss: 3.4138e-04 - val_accuracy: 1.0000
Epoch 1341/1500
4/4 [==============================] - 0s 27ms/step - loss: 3.7677e-05 - accuracy: 1.000

4/4 [==============================] - 0s 27ms/step - loss: 3.3051e-05 - accuracy: 1.0000 - val_loss: 3.3013e-04 - val_accuracy: 1.0000
Epoch 1390/1500
4/4 [==============================] - 0s 29ms/step - loss: 3.2772e-05 - accuracy: 1.0000 - val_loss: 3.0928e-04 - val_accuracy: 1.0000
Epoch 1391/1500
4/4 [==============================] - 0s 29ms/step - loss: 3.2762e-05 - accuracy: 1.0000 - val_loss: 2.9816e-04 - val_accuracy: 1.0000
Epoch 1392/1500
4/4 [==============================] - 0s 26ms/step - loss: 3.2754e-05 - accuracy: 1.0000 - val_loss: 3.0853e-04 - val_accuracy: 1.0000
Epoch 1393/1500
4/4 [==============================] - 0s 27ms/step - loss: 3.2460e-05 - accuracy: 1.0000 - val_loss: 3.2369e-04 - val_accuracy: 1.0000
Epoch 1394/1500
4/4 [==============================] - 0s 35ms/step - loss: 3.2549e-05 - accuracy: 1.0000 - val_loss: 3.2548e-04 - val_accuracy: 1.0000
Epoch 1395/1500
4/4 [==============================] - 0s 27ms/step - loss: 3.2463e-05 - accuracy: 1.000

4/4 [==============================] - 0s 29ms/step - loss: 2.8566e-05 - accuracy: 1.0000 - val_loss: 2.8975e-04 - val_accuracy: 1.0000
Epoch 1444/1500
4/4 [==============================] - 0s 24ms/step - loss: 2.8391e-05 - accuracy: 1.0000 - val_loss: 2.7797e-04 - val_accuracy: 1.0000
Epoch 1445/1500
4/4 [==============================] - 0s 32ms/step - loss: 2.8198e-05 - accuracy: 1.0000 - val_loss: 2.7214e-04 - val_accuracy: 1.0000
Epoch 1446/1500
4/4 [==============================] - 0s 25ms/step - loss: 2.8012e-05 - accuracy: 1.0000 - val_loss: 2.7361e-04 - val_accuracy: 1.0000
Epoch 1447/1500
4/4 [==============================] - 0s 27ms/step - loss: 2.8493e-05 - accuracy: 1.0000 - val_loss: 2.8506e-04 - val_accuracy: 1.0000
Epoch 1448/1500
4/4 [==============================] - 0s 32ms/step - loss: 2.8424e-05 - accuracy: 1.0000 - val_loss: 2.6629e-04 - val_accuracy: 1.0000
Epoch 1449/1500
4/4 [==============================] - 0s 32ms/step - loss: 2.7996e-05 - accuracy: 1.000

4/4 [==============================] - 0s 35ms/step - loss: 2.4595e-05 - accuracy: 1.0000 - val_loss: 2.3884e-04 - val_accuracy: 1.0000
Epoch 1498/1500
4/4 [==============================] - 0s 27ms/step - loss: 2.4486e-05 - accuracy: 1.0000 - val_loss: 2.5493e-04 - val_accuracy: 1.0000
Epoch 1499/1500
4/4 [==============================] - 0s 27ms/step - loss: 2.4394e-05 - accuracy: 1.0000 - val_loss: 2.4844e-04 - val_accuracy: 1.0000
Epoch 1500/1500
4/4 [==============================] - 0s 27ms/step - loss: 2.4252e-05 - accuracy: 1.0000 - val_loss: 2.3995e-04 - val_accuracy: 1.0000


In [56]:
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

In [57]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test_oh, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 1.0


In [58]:
#y_tilde = model.predict(x_test)
#for i in range(len(x_test)):
#  print(i, "Prediction: {}, True label: {}".format(y_tilde[i], y_test[i]))

# **<font color='navy'>Bit Flip vs Phase Flip vs Bit-Phase Flip</font>**

In [59]:
# Generate data
num_samples = 1200
data = generate_Haar_data(1, num_samples)

In [76]:
# Compute the noise-free density matrices and Bloch vectors
density_matrix_noise_free = [*map(get_density_matrix, data)]
bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

In [60]:
# Apply noise 
density_matrix_with_noise = []
for i in range(400):
   single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'bit', 0.1)
   density_matrix_with_noise.append(single_data_with_noise)

for i in range(400, 800):
   single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
   density_matrix_with_noise.append(single_data_with_noise)

for i in range(800, num_samples):
    single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'bp', 0.2)
    density_matrix_with_noise.append(single_data_with_noise)

# Compute the noisy Bloch vectors
bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]

# Concatenate the noisy and noise-free Bloch vectors
complete_vectors = []
for i in range(num_samples):
   element = np.concatenate((bloch_vectors_with_noise[i], bloch_vectors_noise_free[i]))
   complete_vectors.append(element)

for i in range(400):
  complete_vectors[i] = np.append(complete_vectors[i], 0) #bit flip error

for i in range(400, 800):
  complete_vectors[i] = np.append(complete_vectors[i], 1) #phase error

for i in range(800, num_samples):
  complete_vectors[i] = np.append(complete_vectors[i], 2)  #bit-phase error

random.shuffle(complete_vectors)
#print(complete_vectors[3])

In [61]:
#Building the dataset

x_dataset = []

for item in complete_vectors:
  element = item[:-1]
  x_dataset.append(element)

#print(x_dataset)

y_dataset = []
for item in complete_vectors:
  element = item[-1:]
  y_dataset.append(element)

#print(y_dataset)

first_slice_value = 960
last_slice_value = 1080

# Splitting the dataset
x_train_list, x_val_list, x_test_list = x_dataset[:first_slice_value], x_dataset[first_slice_value:last_slice_value], x_dataset[last_slice_value:]

# Convert to tensors
x_train = tf.convert_to_tensor(x_train_list)
x_val = tf.convert_to_tensor(x_val_list)
x_test = tf.convert_to_tensor(x_test_list)

# Defining the labels 
y_label_train, y_label_val, y_label_test = y_dataset[:first_slice_value], y_dataset[first_slice_value:last_slice_value], y_dataset[last_slice_value:]

# Convert to tensors
y_train = tf.convert_to_tensor(y_label_train)
y_val = tf.convert_to_tensor(y_label_val)
y_test = tf.convert_to_tensor(y_label_test)

# Define the one-hot vectors
y_train_oh = tf.keras.utils.to_categorical(y_train, 3)
y_val_oh = tf.keras.utils.to_categorical(y_val, 3)
y_test_oh = tf.keras.utils.to_categorical(y_test, 3)

In [62]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),  
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),   
  tf.keras.layers.Dense(3, activation = 'softmax')
  ])

In [63]:
# Compile model
adam_opt = tf.optimizers.Adam(0.00007)
model.compile(loss='categorical_crossentropy',
             optimizer=adam_opt,
             metrics=['accuracy'])

In [64]:
#Train Model
batch_size = 240
epochs = 5000 

history = model.fit(x_train,
         y_train_oh,
         batch_size=batch_size,
         epochs=epochs,
         validation_data=(x_val, y_val_oh))

Epoch 1/5000
4/4 [==============================] - 2s 141ms/step - loss: 1.1082 - accuracy: 0.3208 - val_loss: 1.0992 - val_accuracy: 0.3750
Epoch 2/5000
4/4 [==============================] - 0s 27ms/step - loss: 1.1060 - accuracy: 0.3208 - val_loss: 1.0989 - val_accuracy: 0.3667
Epoch 3/5000
4/4 [==============================] - 0s 27ms/step - loss: 1.1040 - accuracy: 0.3292 - val_loss: 1.0986 - val_accuracy: 0.3500
Epoch 4/5000
4/4 [==============================] - 0s 27ms/step - loss: 1.1020 - accuracy: 0.3427 - val_loss: 1.0982 - val_accuracy: 0.3583
Epoch 5/5000
4/4 [==============================] - 0s 29ms/step - loss: 1.1002 - accuracy: 0.3479 - val_loss: 1.0979 - val_accuracy: 0.3417
Epoch 6/5000
4/4 [==============================] - 0s 30ms/step - loss: 1.0984 - accuracy: 0.3635 - val_loss: 1.0977 - val_accuracy: 0.3333
Epoch 7/5000
4/4 [==============================] - 0s 35ms/step - loss: 1.0966 - accuracy: 0.3646 - val_loss: 1.0974 - val_accuracy: 0.3083
Epoch 8/5000

Epoch 59/5000
4/4 [==============================] - 0s 29ms/step - loss: 1.0170 - accuracy: 0.5531 - val_loss: 1.0573 - val_accuracy: 0.4833
Epoch 60/5000
4/4 [==============================] - 0s 27ms/step - loss: 1.0150 - accuracy: 0.5583 - val_loss: 1.0557 - val_accuracy: 0.4833
Epoch 61/5000
4/4 [==============================] - 0s 27ms/step - loss: 1.0130 - accuracy: 0.5615 - val_loss: 1.0542 - val_accuracy: 0.4833
Epoch 62/5000
4/4 [==============================] - 0s 24ms/step - loss: 1.0109 - accuracy: 0.5635 - val_loss: 1.0525 - val_accuracy: 0.4917
Epoch 63/5000
4/4 [==============================] - 0s 29ms/step - loss: 1.0088 - accuracy: 0.5667 - val_loss: 1.0507 - val_accuracy: 0.5000
Epoch 64/5000
4/4 [==============================] - 0s 29ms/step - loss: 1.0067 - accuracy: 0.5719 - val_loss: 1.0488 - val_accuracy: 0.5000
Epoch 65/5000
4/4 [==============================] - 0s 29ms/step - loss: 1.0045 - accuracy: 0.5740 - val_loss: 1.0470 - val_accuracy: 0.5083
Epoch 

4/4 [==============================] - 0s 27ms/step - loss: 0.8266 - accuracy: 0.7479 - val_loss: 0.8819 - val_accuracy: 0.6750
Epoch 117/5000
4/4 [==============================] - 0s 27ms/step - loss: 0.8215 - accuracy: 0.7573 - val_loss: 0.8762 - val_accuracy: 0.6917
Epoch 118/5000
4/4 [==============================] - 0s 29ms/step - loss: 0.8163 - accuracy: 0.7646 - val_loss: 0.8712 - val_accuracy: 0.7000
Epoch 119/5000
4/4 [==============================] - 0s 29ms/step - loss: 0.8110 - accuracy: 0.7677 - val_loss: 0.8664 - val_accuracy: 0.7083
Epoch 120/5000
4/4 [==============================] - 0s 27ms/step - loss: 0.8055 - accuracy: 0.7719 - val_loss: 0.8609 - val_accuracy: 0.7167
Epoch 121/5000
4/4 [==============================] - 0s 32ms/step - loss: 0.8002 - accuracy: 0.7760 - val_loss: 0.8547 - val_accuracy: 0.7417
Epoch 122/5000
4/4 [==============================] - 0s 29ms/step - loss: 0.7946 - accuracy: 0.7854 - val_loss: 0.8492 - val_accuracy: 0.7250
Epoch 123/5000

4/4 [==============================] - 0s 29ms/step - loss: 0.4753 - accuracy: 0.9531 - val_loss: 0.5122 - val_accuracy: 0.9417
Epoch 174/5000
4/4 [==============================] - 0s 29ms/step - loss: 0.4695 - accuracy: 0.9594 - val_loss: 0.5071 - val_accuracy: 0.9417
Epoch 175/5000
4/4 [==============================] - 0s 27ms/step - loss: 0.4635 - accuracy: 0.9594 - val_loss: 0.5005 - val_accuracy: 0.9417
Epoch 176/5000
4/4 [==============================] - 0s 27ms/step - loss: 0.4573 - accuracy: 0.9604 - val_loss: 0.4933 - val_accuracy: 0.9417
Epoch 177/5000
4/4 [==============================] - 0s 32ms/step - loss: 0.4516 - accuracy: 0.9594 - val_loss: 0.4864 - val_accuracy: 0.9417
Epoch 178/5000
4/4 [==============================] - 0s 27ms/step - loss: 0.4457 - accuracy: 0.9604 - val_loss: 0.4807 - val_accuracy: 0.9417
Epoch 179/5000
4/4 [==============================] - 0s 29ms/step - loss: 0.4400 - accuracy: 0.9677 - val_loss: 0.4750 - val_accuracy: 0.9417
Epoch 180/5000

4/4 [==============================] - 0s 32ms/step - loss: 0.2191 - accuracy: 0.9948 - val_loss: 0.2444 - val_accuracy: 0.9750
Epoch 231/5000
4/4 [==============================] - 0s 29ms/step - loss: 0.2162 - accuracy: 0.9948 - val_loss: 0.2411 - val_accuracy: 0.9833
Epoch 232/5000
4/4 [==============================] - 0s 27ms/step - loss: 0.2134 - accuracy: 0.9948 - val_loss: 0.2378 - val_accuracy: 0.9833
Epoch 233/5000
4/4 [==============================] - 0s 29ms/step - loss: 0.2106 - accuracy: 0.9948 - val_loss: 0.2345 - val_accuracy: 0.9917
Epoch 234/5000
4/4 [==============================] - 0s 32ms/step - loss: 0.2078 - accuracy: 0.9948 - val_loss: 0.2317 - val_accuracy: 0.9917
Epoch 235/5000
4/4 [==============================] - 0s 27ms/step - loss: 0.2053 - accuracy: 0.9948 - val_loss: 0.2296 - val_accuracy: 0.9833
Epoch 236/5000
4/4 [==============================] - 0s 27ms/step - loss: 0.2024 - accuracy: 0.9948 - val_loss: 0.2264 - val_accuracy: 0.9833
Epoch 237/5000

...

4/4 [==============================] - 0s 35ms/step - loss: 5.9181e-07 - accuracy: 1.0000 - val_loss: 0.0202 - val_accuracy: 0.9917
Epoch 4731/5000
4/4 [==============================] - 0s 29ms/step - loss: 5.8933e-07 - accuracy: 1.0000 - val_loss: 0.0201 - val_accuracy: 0.9917
Epoch 4732/5000
4/4 [==============================] - 0s 24ms/step - loss: 5.9392e-07 - accuracy: 1.0000 - val_loss: 0.0200 - val_accuracy: 0.9917
Epoch 4733/5000
4/4 [==============================] - 0s 29ms/step - loss: 5.8721e-07 - accuracy: 1.0000 - val_loss: 0.0201 - val_accuracy: 0.9917
Epoch 4734/5000
4/4 [==============================] - 0s 24ms/step - loss: 5.9069e-07 - accuracy: 1.0000 - val_loss: 0.0201 - val_accuracy: 0.9917
Epoch 4735/5000
4/4 [==============================] - 0s 27ms/step - loss: 5.8871e-07 - accuracy: 1.0000 - val_loss: 0.0201 - val_accuracy: 0.9917
Epoch 4736/5000
4/4 [==============================] - 0s 32ms/step - loss: 5.8846e-07 - accuracy: 1.0000 - val_loss: 0.0206 - v

4/4 [==============================] - 0s 24ms/step - loss: 5.2712e-07 - accuracy: 1.0000 - val_loss: 0.0206 - val_accuracy: 0.9917
Epoch 4786/5000
4/4 [==============================] - 0s 24ms/step - loss: 5.3308e-07 - accuracy: 1.0000 - val_loss: 0.0212 - val_accuracy: 0.9917
Epoch 4787/5000
4/4 [==============================] - 0s 29ms/step - loss: 5.3035e-07 - accuracy: 1.0000 - val_loss: 0.0212 - val_accuracy: 0.9917
Epoch 4788/5000
4/4 [==============================] - 0s 32ms/step - loss: 5.2935e-07 - accuracy: 1.0000 - val_loss: 0.0207 - val_accuracy: 0.9917
Epoch 4789/5000
4/4 [==============================] - 0s 29ms/step - loss: 5.2612e-07 - accuracy: 1.0000 - val_loss: 0.0205 - val_accuracy: 0.9917
Epoch 4790/5000
4/4 [==============================] - 0s 24ms/step - loss: 5.2240e-07 - accuracy: 1.0000 - val_loss: 0.0207 - val_accuracy: 0.9917
Epoch 4791/5000
4/4 [==============================] - 0s 24ms/step - loss: 5.3320e-07 - accuracy: 1.0000 - val_loss: 0.0202 - v

4/4 [==============================] - 0s 32ms/step - loss: 4.7372e-07 - accuracy: 1.0000 - val_loss: 0.0208 - val_accuracy: 0.9917
Epoch 4841/5000
4/4 [==============================] - 0s 32ms/step - loss: 4.7410e-07 - accuracy: 1.0000 - val_loss: 0.0208 - val_accuracy: 0.9917
Epoch 4842/5000
4/4 [==============================] - 0s 32ms/step - loss: 4.7099e-07 - accuracy: 1.0000 - val_loss: 0.0209 - val_accuracy: 0.9917
Epoch 4843/5000
4/4 [==============================] - 0s 35ms/step - loss: 4.6876e-07 - accuracy: 1.0000 - val_loss: 0.0210 - val_accuracy: 0.9917
Epoch 4844/5000
4/4 [==============================] - 0s 27ms/step - loss: 4.7223e-07 - accuracy: 1.0000 - val_loss: 0.0208 - val_accuracy: 0.9917
Epoch 4845/5000
4/4 [==============================] - 0s 32ms/step - loss: 4.7198e-07 - accuracy: 1.0000 - val_loss: 0.0210 - val_accuracy: 0.9917
Epoch 4846/5000
4/4 [==============================] - 0s 32ms/step - loss: 4.6938e-07 - accuracy: 1.0000 - val_loss: 0.0210 - v

4/4 [==============================] - 0s 29ms/step - loss: 4.2728e-07 - accuracy: 1.0000 - val_loss: 0.0214 - val_accuracy: 0.9917
Epoch 4896/5000
4/4 [==============================] - 0s 29ms/step - loss: 4.2517e-07 - accuracy: 1.0000 - val_loss: 0.0213 - val_accuracy: 0.9917
Epoch 4897/5000
4/4 [==============================] - 0s 27ms/step - loss: 4.2877e-07 - accuracy: 1.0000 - val_loss: 0.0217 - val_accuracy: 0.9917
Epoch 4898/5000
4/4 [==============================] - 0s 24ms/step - loss: 4.3548e-07 - accuracy: 1.0000 - val_loss: 0.0208 - val_accuracy: 0.9917
Epoch 4899/5000
4/4 [==============================] - 0s 24ms/step - loss: 4.2517e-07 - accuracy: 1.0000 - val_loss: 0.0207 - val_accuracy: 0.9917
Epoch 4900/5000
4/4 [==============================] - 0s 27ms/step - loss: 4.2381e-07 - accuracy: 1.0000 - val_loss: 0.0217 - val_accuracy: 0.9917
Epoch 4901/5000
4/4 [==============================] - 0s 24ms/step - loss: 4.2468e-07 - accuracy: 1.0000 - val_loss: 0.0219 - v

4/4 [==============================] - 0s 24ms/step - loss: 3.8643e-07 - accuracy: 1.0000 - val_loss: 0.0217 - val_accuracy: 0.9917
Epoch 4951/5000
4/4 [==============================] - 0s 24ms/step - loss: 3.8395e-07 - accuracy: 1.0000 - val_loss: 0.0218 - val_accuracy: 0.9917
Epoch 4952/5000
4/4 [==============================] - 0s 24ms/step - loss: 3.8345e-07 - accuracy: 1.0000 - val_loss: 0.0217 - val_accuracy: 0.9917
Epoch 4953/5000
4/4 [==============================] - 0s 24ms/step - loss: 3.8494e-07 - accuracy: 1.0000 - val_loss: 0.0219 - val_accuracy: 0.9917
Epoch 4954/5000
4/4 [==============================] - 0s 24ms/step - loss: 3.7948e-07 - accuracy: 1.0000 - val_loss: 0.0219 - val_accuracy: 0.9917
Epoch 4955/5000
4/4 [==============================] - 0s 24ms/step - loss: 3.7923e-07 - accuracy: 1.0000 - val_loss: 0.0217 - val_accuracy: 0.9917
Epoch 4956/5000
4/4 [==============================] - 0s 24ms/step - loss: 3.8457e-07 - accuracy: 1.0000 - val_loss: 0.0213 - v

In [65]:
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

In [66]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test_oh, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 1.0


In [67]:
#y_tilde = model.predict(x_test)
#for i in range(len(x_test)):
#  print(i, "Prediction: {}, True label: {}".format(y_tilde[i], y_test[i]))